# 3A: Genome-scale Modeling
## This notebook shows the code and calculations used to determine:
### - Model changes for all simulations (Notebook 3B).


In [ ]:
import cobra
from cobra.core.metabolite import elements_and_molecular_weights
elements_and_molecular_weights['R']=0.0
elements_and_molecular_weights['Z']=0.0
import fcns.modeling_functions as fcns

## Model set up
### This code shows:
#### 1) Changes to the base model iLB1034
#### 2) New reactions for including PAM data
#### 3) D1 protein repair
#### 4) Biomass composition differences between low and high light

In [ ]:
## 1) Changes to the base model iLB1034
model = cobra.io.load_json_model('Model_iLB1034.json')
solution = model.optimize()
print(solution)

In [ ]:
## Add NGAM reaction to the model:
rxn = model.reactions.AOX_m.copy()
rxn.id = 'NGAM'
rxn.name = 'Non-growth associated maintenance'
model.add_reaction(rxn)
model.repair()
print(rxn)

In [ ]:
# PSI energy transfer
m1 = model.metabolites.photon_h
m2 = m1.copy()
m2.id = 'photon_PSI_u'
m2.compartment = 'u'
m2.name = 'Photon: PSI'

rxn = model.reactions.PHOt_h.copy()
m3 = model.metabolites.photon_c

rxn.id = 'PHO_PSIt_u'
rxn.name = 'Photon transport: LHC to PSI'
rxn.add_metabolites({m1:-2.,
                    m3:1.,
                    m2:1.})
model.add_reaction(rxn)
model.repair()
print(rxn)

# Change PSI reaction to include above changes
rxn = model.reactions.PSI_u
m1 = model.metabolites.photon_h
m2 = model.metabolites.photon_PSI_u

rxn.add_metabolites({m1:2.,
                    m2:-2.})

model.repair()
print(rxn)

In [ ]:
## PSII energy transfer
m1 = model.metabolites.photon_h
m2 = m1.copy()
m2.id = 'photon_YII_u'
m2.compartment = 'u'
m2.name = 'Photon: Photochemical quenching'

m3 = m1.copy()
m3.id = 'photon_YNPQ_u'
m3.compartment = 'u'
m3.name = 'Photon: Regulated non-photochemical quenching'

m4 = m1.copy()
m4.id = 'photon_YNO_u'
m4.compartment = 'u'
m4.name = 'Photon: Unregulated non-photochemical quenching'

m6 = m1.copy()
m6.id = 'photon_1-FvFm_u'
m6.compartment = 'u'
m6.name = 'Photon: Lost in pigment bed (1-Fv/Fm)'

rxn = model.reactions.PHOt_h.copy()
m5 = model.metabolites.photon_c

rxn.id = 'PHO_PSIIt_u'
rxn.name = 'Photon transport: LHC to PSII'
rxn.add_metabolites({m1:-2., ## Dummy fractions in here now, will be PAM data
                    m3:0.3,
                    m2:0.3,
                    m4:0.1,
                    m5:1.,
                    m6:0.3})
model.add_reaction(rxn)
model.repair()
print(rxn)

# Change PSII reaction to include above changes
rxn = model.reactions.PSII_u
m1 = model.metabolites.photon_h
m2 = model.metabolites.photon_YII_u

rxn.add_metabolites({m1:4.,
                    m2:-4.})

model.repair()
print(rxn)

In [ ]:
## Demand reactions to remove all non-photochemistry
### Y(NPQ)
rxn = model.reactions.DM_biomass_c.copy()
rxn.id = 'DM_photon_YNPQ_u'
rxn.name = 'Demand Reaction: NPQ quenching as heat'
m1 = model.metabolites.biomass_c
m2 = model.metabolites.photon_YNPQ_u

rxn.add_metabolites({m1:1.,
                    m2:-1.})

model.add_reaction(rxn)
model.repair()
print(rxn)

### 1-Fv/Fm
rxn = model.reactions.DM_biomass_c.copy()
rxn.id = 'DM_photon_1-FvFm_u'
rxn.name = 'Demand Reaction: Excitation energy lost upstream of PSII'

m1 = model.metabolites.biomass_c
m2 = model.metabolites.get_by_id('photon_1-FvFm_u')

rxn.add_metabolites({m1:1.,
                    m2:-1.})

model.add_reaction(rxn)
model.repair()
print(rxn)

### Y(NO)
rxn = model.reactions.DM_biomass_c.copy()
rxn.id = 'DM_photon_YNO_u'
rxn.name = 'Demand Reaction: NO quenching unregulated losses'

m1 = model.metabolites.biomass_c
m2 = model.metabolites.photon_YNO_u

rxn.add_metabolites({m1:1.,
                    m2:-1.})

model.add_reaction(rxn)
model.repair()
print(rxn)

# Non-photochemical loss of photons
from cobra import Metabolite
reaction = model.reactions.DM_biomass_c.copy()
reaction.id = 'DM_photon_c'
reaction.name = 'Demand Reaction: Photon'
m1 = model.metabolites.biomass_c
m2 = model.metabolites.photon_c

reaction.add_metabolites({m1:1,
                         m2:-1})
reaction.upper_bound = 0.
reaction.lower_bound = 0.

model.add_reaction(reaction)
model.repair()


In [ ]:
## New reactions required for GTP in the plastid
### GTP transporter to get into plastid for plastid protein biogenesis
rxn = model.reactions.GTPt_h
rxn.name = 'GDP/GTP antiport, chloroplast'
m1 = model.metabolites.gdp_c.copy()
m1.id = 'gdp_h'
m1.compartment = 'h'
m2 = model.metabolites.gdp_c

rxn.add_metabolites({m1:-1.,
                    m2:1.})
rxn.lower_bound = -1000.

print(rxn.id)
print(rxn.check_mass_balance())
print(rxn.reaction)

### ATPt_h reversible
model.reactions.ATPt_h.lower_bound = -1000.


### NDK chloroplast
rxn = model.reactions.ATGD_c.copy()
rxn.id = 'NDPK1_h'
rxn._compartments = 'h'
rxn.name = 'Nucleoside-diphosphate kinase (ATP:GDP)'
rxn.gene_reaction_rule= 'Phatr3_EG00931'

m1 = model.metabolites.atp_c
m2 = model.metabolites.gdp_c
m3 = model.metabolites.adp_c
m4 = model.metabolites.gtp_c

m5 = model.metabolites.atp_h
m6 = model.metabolites.gdp_h
m7 = model.metabolites.adp_h
m8 = model.metabolites.gtp_h

rxn.add_metabolites({m1:1.,
                    m2:1.,
                    m3:-1.,
                    m4:-1.,
                    m5:-1.,
                    m6:-1.,
                    m7:1.,
                    m8:1.})
rxn.lower_bound = -1000.
model.add_reaction(rxn)
model.repair()

print(rxn.id)
print(rxn.check_mass_balance())
print(rxn.reaction)

In [ ]:
#### D1 subunit repair reaction
rxn = model.reactions.DM_biomass_c.copy()
rxn.id = 'NGAM_D1_u'
rxn.name = 'PSII D1 (PsbA) repair'


mX = model.metabolites.biomass_c
m1 = model.metabolites.h_h
m2 = model.metabolites.atp_h
m3 = model.metabolites.gtp_h
m4 = model.metabolites.h2o_h
m5 = model.metabolites.adp_h
m6 = model.metabolites.gdp_h
m7 = model.metabolites.pi_h


rxn.add_metabolites({mX:1.,
                    m1:1080.,
                    m2:-360., # 1 ATP per amino acid polymerized
                    m3:-720., # 2 GTP per amino acid polymerized
                    m4:-1080.,
                    m5:360.,
                    m6:720.,
                    m7:1080.})


model.add_reaction(rxn)
model.repair()
print(rxn.id)
print(rxn.check_mass_balance())
print(rxn.reaction)

In [ ]:
# Load the media composition
EXrxn = ['EX_co2_e','EX_hco3_e','EX_no2_e','EX_no3_e','EX_nh4_e','EX_biotin_e','EX_fe2_e','EX_h_e','EX_h2o_e','EX_o2_e','EX_pi_e','EX_na1_e','EX_so4_e','EX_hso3_e','EX_mg2_e','EX_glyclt_e','EX_selt_e','EX_glc_e','EX_cl_e','EX_thm_e','EX_h2_e','EX_fol_e','EX_co_e','EX_cyan_e','EX_cynt_e','EX_tcynt_e','EX_lac_d_e','EX_etoh_e']
EXub = [1000,1000,0,0,1000,0,1000,1000,1000,1000,1000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
EXlb = [0,-1000,0,-1000,0,-1000,-1000,-1000,-1000,-1000,-1000,-1000,-1000,0,-1000,0,0,0,-1000,-1000,0,0,0,0,0,0,0,0]
for i in range(len(EXrxn)):
    rx = EXrxn[i]
    model.reactions.get_by_id(rx).lower_bound = EXlb[i]
    model.reactions.get_by_id(rx).upper_bound = EXub[i]

In [ ]:
# Initial constraints 
# Turn off reactions that cause biologically unrealistic constraints
rxn =  ['DIATOXEPOX_h','VIOXANDE_h','ANTHXANDE_h']
for i in range(len(rxn)):
    rx = rxn[i]
    model.reactions.get_by_id(rx).lower_bound = 0
    model.reactions.get_by_id(rx).upper_bound = 0
    
# Remove constraints on CEF_h (will be added during simulations)
model.reactions.CEF_h.lower_bound = 0.0
model.reactions.CEF_h.upper_bound = 1000

# flux only constraints
model.reactions.PDYXPT_c.upper_bound = 0.
model.reactions.PDYXPT_c.lower_bound = 0.
model.reactions.PYDXDH_c.upper_bound = 0.
model.reactions.PYDXDH_c.lower_bound = 0.
model.reactions.ATPM_c.upper_bound = 0.
model.reactions.ATPM_c.lower_bound = 0.
model.reactions.ATPM_h.upper_bound = 0.
model.reactions.ATPM_h.lower_bound = 0.
model.reactions.ATPM_m.upper_bound = 0.
model.reactions.ATPM_m.lower_bound = 0.

# Set AEF to Ballieu 2015 Nature paper

model.reactions.PTOX_h.upper_bound = 0.
model.reactions.MEHLER_h.upper_bound = 0.

# Changing the bounds to umol instead of mmol
for r in model.reactions:
    r.lower_bound = r.lower_bound*1000.
    r.upper_bound = r.upper_bound*1000.

## Turn off all other biomass equations
model.reactions.bof_c_accumulation_c = 0.
model.reactions.bof_cell_division_c.upper_bound = 0.
model.reactions.bof_dark_c.upper_bound = 0.

In [ ]:
## High light biomass composition
## Total Pigments
## HL 2.2% total mass
pigment_mass = {'caro_h':0.006,
               'cholphya_h':0.142,
               'cholphyc1_h':0.009,
               'cholphyc2_h':0.015,
               'diadinx_h':0.02,
               'diatox_h':0.004,
               'fxanth_h':0.024,
               'chlda_h':0.0}
pigment_data = dict()
for pigm,mass in pigment_mass.items():
    met_obj = model.metabolites.get_by_id(pigm)
    met_stoich = mass/met_obj.formula_weight
    pigment_data[met_obj]=met_stoich*-1*1000.

HL_model = fcns.update_biomass(model,'biomass_pigm_h',pigment_data,'biomass_pigm_h')    

## Total Biomass
bof_ratios = {'biomass_carb_c':0.060,
               'biomass_dna_c':0.0032,
               'biomass_mem_lipids_c':0.030,
               'biomass_pigm_h':0.022, 
               'biomass_plastid_lip_h':0.037,
               'biomass_pro_c':0.70,
               'biomass_rna_c':0.027,
               'carbon_storage_c':0.10}

bof_data = dict()

for bofcmp,percent in bof_ratios.items():
    met_obj = model.metabolites.get_by_id(bofcmp)
    bof_data[met_obj]=percent*-1

HL_model = fcns.update_biomass(HL_model,'bof_c',bof_ratios,'biomass_c')

cobra.io.save_json_model(HL_model,'HL_base_iLB1035.json')


In [ ]:
## Low light biomass composition
## Total Pigments
## LL 4.8%
pigment_mass = {'caro_h':0.012,
               'cholphya_h':0.384,
               'cholphyc1_h':0.053,
               'cholphyc2_h':0.089,
               'diadinx_h':0.030,
               'diatox_h':0.008,
               'fxanth_h':0.165,
               'chlda_h':0.0}
pigment_data = dict()
for pigm,mass in pigment_mass.items():
    met_obj = model.metabolites.get_by_id(pigm)
    met_stoich = mass/met_obj.formula_weight
    pigment_data[met_obj]=met_stoich*-1*1000.

LL_model = fcns.update_biomass(model,'biomass_pigm_h',pigment_data,'biomass_pigm_h')    

## Total Biomass
bof_ratios = {'biomass_carb_c':0.056,
               'biomass_dna_c':0.0030,
               'biomass_mem_lipids_c':0.028,
               'biomass_pigm_h':0.048,
               'biomass_plastid_lip_h':0.070,
               'biomass_pro_c':0.64,
               'biomass_rna_c':0.025,
               'carbon_storage_c':0.10}

bof_data = dict()

for bofcmp,percent in bof_ratios.items():
    met_obj = model.metabolites.get_by_id(bofcmp)
    bof_data[met_obj]=percent*-1

LL_model = fcns.update_biomass(HL_model,'bof_c',bof_ratios,'biomass_c')

cobra.io.save_json_model(HL_model,'LL_base_iLB1035.json')